The main purpose of this notebook is the coursera capstone project

In [4]:
import pandas as pd
import numpy as np
import requests

In [3]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


# Introduction/Business Problem

We are a consulting society specialized in datascience applied to business. Our client ask us where he should install his society. He chose Paris but didn't decide now the neighborhood. We need to make a study to find the neighborhood that suits the best. This project is very important for my client because he would have more possibilities of succeed if the neighborhood has been chosen wisely. In the other hand, a bad choice would make a bad start and would be really detrimental for the company. 
We have to compare different neighborhood, and find the ones that fits the best with the client's expectation. The client's business will be a shop specialized in Japanese food.

# Data

We will massively use Foursquare in this analysis. We make the assumption that people interessed in Japanese food in Paris are also interessed in other Asian food. We will then focus on data relative to Asian food in Paris. It will be shop, restaurant.. We will also cluster the different neighborhood in Paris and validate our analysis based on the clustering with data visualisation. That means that we need neighborhood venues. Then we will choose the neighborhood that fits the best with all information that we will have gathered. Finally, we will get all Paris neighborhood from a csv file. This is open data 

https://opendata.paris.fr/explore/dataset/quartier_paris/export/?dataChart=eyJxdWVyaWVzIjpbeyJjb25maWciOnsiZGF0YXNldCI6InF1YXJ0aWVyX3BhcmlzIiwib3B0aW9ucyI6e319LCJjaGFydHMiOlt7ImFsaWduTW9udGgiOnRydWUsInR5cGUiOiJjb2x1bW4iLCJmdW5jIjoiQVZHIiwieUF4aXMiOiJuX3NxX3F1Iiwic2NpZW50aWZpY0Rpc3BsYXkiOnRydWUsImNvbG9yIjoiIzAwMzM2NiJ9XSwieEF4aXMiOiJsX3F1IiwibWF4cG9pbnRzIjo1MCwic29ydCI6IiJ9XSwidGltZXNjYWxlIjoiIiwiZGlzcGxheUxlZ2VuZCI6dHJ1ZSwiYWxpZ25Nb250aCI6dHJ1ZX0%3D

In [2]:
# @hidden_cell
CLIENT_ID = 'ODSQTRMSAFMCAM53ZP5ZJ4Y0DJECUY1DJQCNL3TEQFX03FO5' # your Foursquare ID
CLIENT_SECRET = 'BUIC2UJZOZ3OKMHHQYSIX3MKUIX0UZKSC2GEYVCQI5YAKPQ4' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

Venue request example

In [5]:
lat=48.8534
lng=2.3488
radius=500

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
results = requests.get(url).json()["response"]['groups'][0]['items']

In [7]:
print(results)

[{'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue': {'id': '4adcda21f964a520f23921e3', 'name': 'Shakespeare & Company', 'location': {'address': '37 rue de la Bûcherie', 'lat': 48.85256839809226, 'lng': 2.3470958322286606, 'labeledLatLngs': [{'label': 'display', 'lat': 48.85256839809226, 'lng': 2.3470958322286606}], 'distance': 155, 'postalCode': '75005', 'cc': 'FR', 'city': 'Paris', 'state': 'Île-de-France', 'country': 'France', 'formattedAddress': ['37 rue de la Bûcherie', '75005 Paris', 'France']}, 'categories': [{'id': '4bf58dd8d48988d114951735', 'name': 'Bookstore', 'pluralName': 'Bookstores', 'shortName': 'Bookstore', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/bookstore_', 'suffix': '.png'}, 'primary': True}], 'photos': {'count': 0, 'groups': []}}, 'referralId': 'e-0-4adcda21f964a520f23921e3-0'}, {'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 't

In [11]:
venues_list=[]
venues_list.append([( 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
nearby_venues.columns = [
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
nearby_venues

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,48.8534,2.3488,Shakespeare & Company,48.852568,2.347096,Bookstore
1,48.8534,2.3488,Tours de la Cathédrale Notre-Dame de Paris,48.853230,2.349207,Scenic Lookout
2,48.8534,2.3488,Parvis Notre-Dame — Place Jean-Paul II,48.853407,2.348456,Plaza
3,48.8534,2.3488,Au Vieux Paris d'Arcole,48.854196,2.350312,French Restaurant
4,48.8534,2.3488,Amorino,48.853012,2.345771,Ice Cream Shop
...,...,...,...,...,...,...
56,48.8534,2.3488,Paris Rendez-Vous,48.856951,2.352914,Souvenir Shop
57,48.8534,2.3488,Le Céleste Gourmand,48.857566,2.350047,Chinese Restaurant
58,48.8534,2.3488,La Frange,48.850214,2.352772,Bar
59,48.8534,2.3488,L'Elica,48.848994,2.347447,Pizza Place


Get all Asian Restaurant in 1km radius from Paris' center

In [22]:
lat=48.8534
lng=2.3488
radius=1000
category="4bf58dd8d48988d142941735"

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng,
            category,
            radius, 
            LIMIT)
results = requests.get(url).json()["response"]['groups'][0]['items']

In [24]:
asian_restaurant_list=[]
asian_restaurant_list.append([( 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

paris_asian_venues = pd.DataFrame([item for asian_restaurant_list in asian_restaurant_list for item in asian_restaurant_list])
paris_asian_venues.columns = [
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
paris_asian_venues

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,48.8534,2.3488,Jixiao's Buns,48.854234,2.343562,Asian Restaurant
1,48.8534,2.3488,Kokonor,48.845544,2.342637,Asian Restaurant
2,48.8534,2.3488,Xin Gainian,48.853982,2.338757,Asian Restaurant
3,48.8534,2.3488,Le Céleste Gourmand,48.857566,2.350047,Chinese Restaurant
4,48.8534,2.3488,Forum Sushi,48.862280,2.350287,Asian Restaurant
...,...,...,...,...,...,...
83,48.8534,2.3488,Saveurs d'Asie,48.850259,2.348774,Vietnamese Restaurant
84,48.8534,2.3488,Sakura,48.861076,2.353815,Sushi Restaurant
85,48.8534,2.3488,La Maison du Dim Sum,48.846009,2.343437,Dim Sum Restaurant
86,48.8534,2.3488,Leader Sushi,48.862233,2.349049,Sushi Restaurant


Get all Bubble tea shop (linked with Asian shop) in 20km radius from Paris' center

In [29]:
lat=48.8534
lng=2.3488
radius=20000
category="52e81612bcbc57f1066b7a0c"

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng,
            category,
            radius, 
            LIMIT)
results = requests.get(url).json()["response"]['groups'][0]['items']

In [30]:
bubble_tea_list=[]
bubble_tea_list.append([( 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

bubble_tea_venues = pd.DataFrame([item for bubble_tea_list in bubble_tea_list for item in bubble_tea_list])
bubble_tea_venues.columns = [
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
bubble_tea_venues

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,48.8534,2.3488,Laïzé 來座 (Laïzé),48.863077,2.355073,Bubble Tea Shop
1,48.8534,2.3488,ZenZoo – Bar à Thé,48.867021,2.336738,Bubble Tea Shop
2,48.8534,2.3488,Le 37m²,48.868636,2.336722,Bubble Tea Shop
3,48.8534,2.3488,Bubbolitas - Bubble Tea Bar,48.859810,2.349812,Bubble Tea Shop
4,48.8534,2.3488,ZenZoo,48.867771,2.337020,Taiwanese Restaurant
5,48.8534,2.3488,TeaYammi,48.866760,2.335194,Bubble Tea Shop
6,48.8534,2.3488,Bubble T. Paris,48.841039,2.329861,Bubble Tea Shop
7,48.8534,2.3488,Herb-Tea Taiwan,48.873801,2.337900,Tea Room
8,48.8534,2.3488,Chatime,48.865320,2.336665,Bubble Tea Shop
9,48.8534,2.3488,Bubbleteacup,48.861968,2.351870,Tea Room


Let's get all Paris neighborhood from the csv file

In [36]:
Paris_neighborhood =  pd.read_csv("quartier_paris.csv", delimiter=';')

In [37]:
Paris_neighborhood

,N_SQ_QU,C_QU,C_QUINSEE,L_QU,C_AR,N_SQ_AR,PERIMETRE,SURFACE,Geometry X Y,Geometry
0,750000036,36,7510904,Rochechouart,9,750000009,2862.450525,5.004354e+05,"48.8798119198,2.344861291","{""type"": ""Polygon"", ""coordinates"": [[[2.349708..."
1,750000047,47,7511203,Bercy,12,750000012,6155.005036,1.902932e+06,"48.8352090499,2.38621008421","{""type"": ""Polygon"", ""coordinates"": [[[2.391141..."
2,750000002,2,7510102,Halles,1,750000001,2606.417128,4.124585e+05,"48.8622891081,2.34489885831","{""type"": ""Polygon"", ""coordinates"": [[[2.349365..."
3,750000026,26,7510702,Invalides,7,750000007,4434.656489,1.073734e+06,"48.8585149661,2.31644474034","{""type"": ""Polygon"", ""coordinates"": [[[2.319011..."
4,750000035,35,7510903,Faubourg-Montmartre,9,750000009,2786.541926,4.173351e+05,"48.8739346918,2.34325257947","{""type"": ""Polygon"", ""coordinates"": [[[2.340255..."
...,...,...,...,...,...,...,...,...,...,...
75,750000017,17,7510501,Saint-Victor,5,750000005,3087.215040,6.041561e+05,"48.8476636355,2.35409315338","{""type"": ""Polygon"", ""coordinates"": [[[2.354921..."
76,750000031,31,7510803,Madeleine,8,750000008,3791.509735,7.607266e+05,"48.871265493,2.32005302865","{""type"": ""Polygon"", ""coordinates"": [[[2.325031..."
77,750000078,78,7512002,Saint-Fargeau,20,750000020,5405.036433,1.486971e+06,"48.8710347391,2.40617153015","{""type"": ""Polygon"", ""coordinates"": [[[2.413813..."
78,750000063,63,7511603,Porte-Dauphine,16,750000016,7447.041546,3.086718e+06,"48.8717656343,2.27191774047","{""type"": ""Polygon"", ""coordinates"": [[[2.270984..."
